In [8]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
from astropy.coordinates import EarthLocation
from astropy.table import Table
from astropy.time import Time
import astropy.units as u
from icecream import ic, install
import numpy as np
import pandas as pd

import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
conn = connect("../../django/TargetDB/db.sqlite3")
html_dir = "html"

session = tlc.ObservingSession(ap.Observer.at_site("LBT"))
session.add_day_range("2025-03-14", "2025-03-18")

phase_defs = [
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.0)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.05)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.18)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.32)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.45)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.55)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.68)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.82)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.95)),
]

pepsi_args = {
    "fiber": "300",
    "cd_blue": 3,
    "cd_red": 6,
    "snr": 100,
    "binocular": True,
    "priority": "(see grid)",
}

name = "LBT Observing List 2025-03-10"
creator = tlc.TargetListCreator(name=name, connection=conn)
creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
    # tlc.add_speckle,
    tlc.add_pepsi,
    tlc.add_lists,
    tlc.add_ephemerides,
    partial(tlc.filter_targets, criteria=lambda df: (df["Num PEPSI"] > 0)),
    partial(tlc.filter_targets, criteria=lambda df: (df["List LBT 2025A"])),
    partial(tlc.concat_dataframe, other_df=pd.read_csv("Calibration Targets.csv")),
    partial(tlc.add_observability, observing_session=session, calc_moon_distance=True),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
    # partial(tlc.filter_targets, criteria=lambda df: (df["Observable Max Alt"] > 40)),
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: np.isnan(df["Teff"])),
    partial(lbt.add_pepsi_params, **pepsi_args),
    partial(tlc.filter_targets, criteria=lambda df: df["PEPSI exp_time"] < 400),
    partial(lbt.assign_rv_standards, science_types=["QuadEB", "SextEB"], rv_standard_type="RV Standard"),
    partial(tlc.add_rv_status, phase_event_defs=phase_defs),
]
tl = creator.calculate(verbose=False)
tl.target_list["PEPSI notes"] = [type.replace("QuadEB", "Science Target") for type in tl.target_list["Target Type"]]
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session)

print(tl.summarize())
print(f"{np.sum(tl.target_list["PEPSI exp_time"])/60:.1f} minutes")
# tl.target_list

LBT Observing List 2025-03-10
27 targets:
      16 QuadEB
       8 RV Standard
       3 Telluric Standard
Column Count (primary, secondary):
    Target: (1, 2)
    Coordinates: (6, 3)
    Count: (1, 0)
    List : (0, 11)
    Observable: (5, 20)
    PEPSI : (3, 6)
Associated tables:
     103 rows,  8 columns: PEPSI
     673 rows,  7 columns: Ephem
       5 rows,  2 columns: Lunar Phases
     103 rows,  5 columns: PEPSI RV Status

58.1 minutes


In [10]:
illumination_categories = [
    ((0.0, 0.4), "Dark"),
    ((0.4, 0.7), "Gray"),
    ((0.7, 1.0), "Bright"),
]

distance_categories = {
    "Dark": [
        ((0, 180), 1),
    ],
    "Gray": [
        ((0, 5), 0.1),
        ((5, 15), 0.75),
        ((15, 180), 1),
    ],
    "Bright": [
        ((0, 15), 0.25),
        ((15, 30), 0.75),
        ((30, 180), 1),
    ],
}

altitude_categories = [
    ((-90, 35), 0),
    ((35, 45), 0.95),
    ((45, 90), 1),
]

phase_scores = {
    "B|B": 0.5,
    "B|R": 0.8,
    "B|T": 0.4,
    "R|B": 0.8,
    "R|R": 0.9,
    "R|T": 1,
    "T|B": 0.4,
    "T|R": 1,
    "T|T": 0.1,
}

science_tl = tl.copy()
science_tl.target_list = science_tl.target_list[science_tl.target_list["Target Type"] == "QuadEB"]
pl = pr.PriorityList(science_tl, session, interval=60 * u.min)
pr.calculate_moon_priority(pl, illumination_categories=illumination_categories, dist_categories=distance_categories)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.calculate_list_priority(pl, "HQND", false_value=0.75)
pr.calculate_phase_priority(pl, phase_defs=phase_defs, phase_categories=phase_scores)
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.3)
pl.categorize_priorities(
    bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"]
)

 '2025-03-14T04:00:00.000000000' '2025-03-14T05:00:00.000000000'
 '2025-03-14T06:00:00.000000000' '2025-03-14T07:00:00.000000000'
 '2025-03-14T08:00:00.000000000' '2025-03-14T09:00:00.000000000'
 '2025-03-14T10:00:00.000000000' '2025-03-14T11:00:00.000000000'
 '2025-03-14T12:00:00.000000000'], obsgeoloc=[( -156086.37580263,  5372484.32928419, 3427908.94705125),
 (-1544631.37136511,  5145741.88700631, 3431321.16440488),
 (-2826762.81938884,  4566423.40536422, 3434486.83506752),
 (-3914629.24804412,  3674223.70861242, 3437189.04976397),
 (-4733690.19817694,  2530276.19915105, 3439242.6547254 ),
 (-5227823.72540227,  1212964.00512233, 3440506.93834325),
 (-5363171.8680748 ,  -187450.82084684, 3440895.27269348),
 (-5130460.59019742, -1575012.04518794, 3440381.04929285),
 (-4545635.23709475, -2854644.16143325, 3438999.50236504),
 (-3648767.96251784, -3938666.95095388, 3436845.29467889),
 (-2501311.99012419, -4752803.31958468, 3434066.03136794)] m, obsgeovel=[(-391.75646354,  -11.99458592,  

In [11]:
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session)
with open("UVa_Multistar.README", "w") as f:
    f.write(readme)

other_files = {
    "LBT Readme": lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session),
    "LBT CSV": open("UVa_Multistar.csv", "r").readlines(),
}

html.render_observing_pages(tl, pl, other_files, "../../Observing Files/LBT Observing 2025-03-14")
# pl.categorical_priorities[0]

In [13]:
import glob
# for categorical_file in glob.glob("*.*"):

for categorical_file in glob.glob("../../Observing Files/LBT Observing 2025-03-14/Categorical Priorities *.html"):

    html.html_to_pdf(categorical_file, categorical_file.replace(".html", ".pdf"))
    print(categorical_file)

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.